## Setup - Import and merge data

In [ ]:
import os
import pandas as pd

raw_bond_data_path = os.environ.get('BOND_DOWNLOAD_PATH')
raw_parcel_data_path = os.environ.get('PARCEL_DOWNLOAD_PATH')

# Convert to pandas DataFrame
bond_data = pd.read_excel(raw_bond_data_path, dtype={'Project: Project Name':str})
print(bond_data.head())

parcel_data = pd.read_excel(raw_parcel_data_path, dtype={'ProjectName':str})
print(parcel_data.head())

  Project Current OID       Project: Project Name                  Address1  \
0              18-92A                Renton Crest       3151 NE 16th Street   
1              19-65A           Polaris at SeaTac  15307 International Blvd   
2              19-74A  Filipino Community Village       5727 37th Avenue S.   
3              19-75A             Confluence, The           3631 S 152nd St   
4              19-77A           Uncle Bob's Place             417 8th Ave S   

      City    Zip County  Count of Sites  Total Building Count  \
0   Renton  98056   King               6                    23   
1   Seatac  98199   King               1                    23   
2  Seattle  98118   King               1                     1   
3  Tukwila  98188   King               1                     1   
4  Seattle  98104   King               1                     1   

   Common Area Setaside  Market Rate Setaside  ...  LIH 80%  Elderly Setaside  \
0                     3                     1  

### Attempting to merge on Project Name and Address columns since that seems to be the only options to join on, but neither option returns any common records

In [ ]:
combined_data_project = pd.merge(bond_data, parcel_data, left_on='Project: Project Name', right_on='ProjectName')
print(combined_data_project.head())

Empty DataFrame
Columns: [Project Current OID, Project: Project Name, Address1, City, Zip, County, Count of Sites, Total Building Count, Common Area Setaside, Market Rate Setaside, Compliance LIH Units, Compliance Units, LIH 30%, LIH 35%, LIH 40%, LIH 45%, LIH 50%, LIH 60%, LIH 80%, Elderly Setaside, Disabled Setaside, Large Household Setaside, Property Management Company, Authorized Signer Account, First Credit Year, Year 15, TC Reg Agreement Exp Date, Monitoring Agencies, ProjectName, TotalUnitsBySize, YearCompleted, Borrower, Address, TotalCityUnitsbySize, OperatingUnits, OperatingCityUnits, WBARS Parcel #s, King County Parcel #s, Comments ]
Index: []

[0 rows x 39 columns]


In [40]:
combined_data_address = pd.merge(bond_data, parcel_data, left_on='Address1', right_on='Address')
print(combined_data_address.head())

Empty DataFrame
Columns: [Project Current OID, Project: Project Name, Address1, City, Zip, County, Count of Sites, Total Building Count, Common Area Setaside, Market Rate Setaside, Compliance LIH Units, Compliance Units, LIH 30%, LIH 35%, LIH 40%, LIH 45%, LIH 50%, LIH 60%, LIH 80%, Elderly Setaside, Disabled Setaside, Large Household Setaside, Property Management Company, Authorized Signer Account, First Credit Year, Year 15, TC Reg Agreement Exp Date, Monitoring Agencies, ProjectName, TotalUnitsBySize, YearCompleted, Borrower, Address, TotalCityUnitsbySize, OperatingUnits, OperatingCityUnits, WBARS Parcel #s, King County Parcel #s, Comments ]
Index: []

[0 rows x 39 columns]


### In looking at the some examples from both files, it looks like the project names in the Bond file are upper and lowercase while it is fully uppercase in the Parcel file. So after converting both of those columns to uppercase, the join results in a dataset with 129 shared project names

In [ ]:
bond_data['Project: Project Name'] = bond_data['Project: Project Name'].str.upper()
parcel_data['ProjectName'] = parcel_data['ProjectName'].str.upper()

combined_data = pd.merge(bond_data, parcel_data, left_on='Project: Project Name', right_on='ProjectName')
print(combined_data.head())
print(len(combined_data))

  Project Current OID       Project: Project Name  \
0              19-74A  FILIPINO COMMUNITY VILLAGE   
1              19-77A           UNCLE BOB'S PLACE   
2              19-72A             WILLOW CROSSING   
3             18-112A               ASPEN TERRACE   
4             18-120A           ENCORE APARTMENTS   

                               Address1     City    Zip County  \
0                   5727 37th Avenue S.  Seattle  98118   King   
1                         417 8th Ave S  Seattle  98104   King   
2  6901 Martin Luther King Jr Way South  Seattle  98118   King   
3                   1717 Belmont Avenue  Seattle  98122   King   
4                        3010 First Ave  Seattle  98121   King   

   Count of Sites  Total Building Count  Common Area Setaside  \
0               1                     1                     1   
1               1                     1                     1   
2               1                     1                     0   
3               1       

### However, I noticed when looking through some of the entries that the addresses did not seem to match up either. Sometimes this was due to differences like AVE vs. Avenue and S vs. South, but there were also some instances of different street numbers and even different street names. The latter differences definitely make me concerned that some properties are not actually the same.

In [ ]:
combined_data[['Project: Project Name','Address','Address1']]

,Project: Project Name,Address,Address1
0,FILIPINO COMMUNITY VILLAGE,5727 37TH AVE S,5727 37th Avenue S.
1,UNCLE BOB'S PLACE,714 S KING ST,417 8th Ave S
2,WILLOW CROSSING,6901 M L KING JR WY S,6901 Martin Luther King Jr Way South
3,ASPEN TERRACE,1723 BELMONT AVE,1717 Belmont Avenue
4,ENCORE APARTMENTS,3010 1ST AVE,3010 First Ave
...,...,...,...
124,COMPASS BROADVIEW,147 N 132ND ST,13047 Greenwood Ave N
125,GARDNER HOUSE,2870 S HANFORD ST,2870 South Hanford St
126,POLARIS AT RAINIER BEACH,9400 RAINIER AVE S,9400 Rainier Avenue S.
127,AN LAC,1253 S JACKSON ST,"1253 S. Jackson Street, Suite A"


### We can also check accuracy by looking at the number of units listed in each dataset. There are only 5 matching projects in this dataset that do not have the same number of units which does give me more confidence that the matches are accurate.

In [ ]:
# create True/False column indicating whether the number of units matches across the two datasets
combined_data['Matching Units'] = combined_data['Compliance Units'] == combined_data['OperatingUnits']

# number of matching projects with different number of units
print(len(combined_data.loc[combined_data['Matching Units']==False,]))
print(combined_data.loc[combined_data['Matching Units']==False,])

5
   Project Current OID             Project: Project Name  \
13             21-115A           ALTAIRE AT JACKSON PARK   
21              23-61A                JUNIPER APARTMENTS   
25              98-30N                    RAVENNA SCHOOL   
65               04-36  1811 EASTLAKE SUPPORTIVE HOUSING   
72              05-92A              CAMBRIDGE APARTMENTS   

                  Address1     City    Zip County  Count of Sites  \
13       14343 15th Ave NE  Seattle  98125   King               1   
21      321 10th Ave South  Seattle  98104   King               1   
25  6545 Ravenna Avenue NE  Seattle  98115   King               1   
65       1811 Eastlake Ave  Seattle  98101   King               1   
72      13030 Linden Ave N  Seattle  98133   King               1   

    Total Building Count  Common Area Setaside  Market Rate Setaside  ...  \
13                     1                     1                     0  ...   
21                     1                     0                     0

### Overall, these two datasets are clearly describing similar projects but are not organized in a way that makes them easy to combine. I would want to check with the owners of both of these datasets to confirm my assumptions about which columns map together (like Compliance Units and Operating Units) and determine if there are any other shared columns like a project ID that might be easier to join on. I would also want to do futher research into the address discrepancies to confirm that the projects in the combined dataset are actually a 100% match

In [58]:
combined_data.to_excel(os.environ.get('DOWNLOAD_PATH'), index=False)